In [1]:
import torch
import transformers
import torch.nn.functional as F


In [2]:

model_name = "/media/cloudbeer/PSSD/huggingface/chatglm-6b"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)

def to_embeddings(model,text):
    input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)
    model_output = model(input_ids, output_hidden_states=True)
    data = (model_output.hidden_states[-1].transpose(0, 1))[0]
    data = F.normalize(torch.mean(data, dim=0), p=2, dim=0)
    return data.tolist()

def model_fn(model_dir):
    pipe = transformers.AutoModel.from_pretrained(model_name,trust_remote_code=True).half()
    pipe.to("cuda")
    return pipe


def predict_fn(data, pipe):
    text = data.pop("text", data)
    type = data.pop("type", 0)

    if type == 0:
        return to_embeddings(pipe, text)
    else:
        response, history = pipe.chat(tokenizer, text, history=[])
        return response

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [3]:
pipe = model_fn(None);


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:

inputs = {
    "text": "段誉是谁？",
    "type": 0
}

res1 = predict_fn(inputs, pipe)

print(res1[0:5], len(res1))

[-0.00203704833984375, 0.00971221923828125, 0.00615692138671875, 0.0003457069396972656, 0.0012903213500976562] 4096


In [5]:
inputs = {
    "text": "段誉是谁？",
    "type": 1
}

res1 = predict_fn(inputs, pipe)

print(res1)

The dtype of attention mask (torch.int64) is not bool


段誉是金庸先生所著武侠小说《天龙八部》中的人物之一，是一个出身名门望派、天赋异禀的武学天才，同时也是一个忠诚正直、善良温和的人。
